# object_detection.ipynb

Detects objects in a folder containing images (*.jpg). 
Results are saved in an SQLite database,

In [17]:
from ultralytics import YOLO
import sqlite3
from icecream import ic
import os
import numpy as np

In [18]:
MODEL_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt'
IMAGE_DIR_PATH = '/home/aubrey/Desktop/Guam07-training-set/rawdatasubset'
OBJECTS_DB_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/objects_subset.sqlite3'

In [19]:
def detect_objects(model_file_path, image_dir_path):
    ic()
    model = YOLO(MODEL_FILE_PATH)
    results = model(
        source=f'{image_dir_path}/*.jpg',
        imgsz=960,
        agnostic_nms=True,
        iou=0.5,
        stream=True,
        )
    return results

In [20]:
rng = np.random.default_rng(42)

def assign_subset(subset_list=['train', 'val', 'test'], probability_list=[0.8, 0.1, 0.1]):
    ''' 
    Returns member of the subset_list at random using probabilities specified in probability_list.
    This function uses a global random number generator (rng).
    Use "rng = np.random.default_rng(42)" to create rng prior to calling the function.
    '''
    r = rng.multinomial(1, probability_list)
    i = np.where(r == 1)[0][0]
    subset = subset_list[i]
    return subset

# for i in range(20):
#     print(i, assign_subset())

In [21]:
def detected_objects_to_sqlite(detected_objects, objects_db_file_path):
    ic()
    if os.path.exists(objects_db_file_path):
        os.remove(OBJECTS_DB_FILE_PATH)

    conn = sqlite3.connect(objects_db_file_path)
    cur = conn.cursor()

    cur.execute('DROP TABLE IF EXISTS detected_objects')
    table = '''
    create table detected_objects (
        imagepath TEXT not null,
        x REAL not null,
        y REAL not null,
        w REAL not null,
        h REAL not null,
        cls INTEGER not null,
        conf REAL not null,
        subset TEXT NOT NULL
    );
    '''
    cur.execute(table)

    for obj in detected_objects:
        imagepath = obj.path
        boxes = obj.boxes.cpu().numpy()
        class_ids = boxes.cls
        confidences = boxes.conf
        xywhns = boxes.xywhn
        n = len(class_ids)
        subset = assign_subset()
        for i in range(n):
            x = xywhns[i][0]    
            y = xywhns[i][1]
            w = xywhns[i][2]
            h = xywhns[i][3]
            cls = int(class_ids[i])
            conf = confidences[i]
            cur.execute(f'INSERT INTO detected_objects VALUES ("{imagepath}",{x},{y},{w},{h},{cls},{conf},"{subset}")')
            conn.commit()
    conn.close()

In [22]:
# MAIN

detected_objects = detect_objects(MODEL_FILE_PATH, IMAGE_DIR_PATH)
detected_objects_to_sqlite(detected_objects, OBJECTS_DB_FILE_PATH)

print('FINISHED')

ic| 2396985220.py:2 in detect_objects() at 11:01:56.193
ic| 1308243588.py:2 in detected_objects_to_sqlite() at 11:01:56.286



image 1/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111515.jpg: 544x960 (no detections), 65.8ms
image 2/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111516.jpg: 544x960 (no detections), 4.1ms
image 3/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111517.jpg: 544x960 (no detections), 3.8ms
image 4/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111518.jpg: 544x960 (no detections), 3.8ms
image 5/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111519.jpg: 544x960 (no detections), 4.0ms
image 6/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111521.jpg: 544x960 (no detections), 3.6ms
image 7/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111522.jpg: 544x960 (no detections), 3.9ms
image 8/1925 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111523.jpg: 544x960 (no detections), 4.1ms
image 